In [72]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import ast

In [73]:
dataframe = pd.read_csv("../Data/data.csv")

In [74]:
dataframe.describe()

,EndOfLifeFramework,ForFrequency,IfFrequency,ForEachFrequency,WhileFrequency,CodeLines,CommentLines,MethodNumber,ClassNumber,InterfaceNumber,InheritanceDeclarations,InheritanceDepth,HttpClientCalls,UsingsNumber
count,506.000000,506.000000,506.000000,506.000000,506.000000,5.060000e+02,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,5.060000e+02,506.000000
mean,4.059289,28.185771,797.958498,85.480237,14.371542,3.038194e+04,4920.847826,618.160079,326.466403,14.132411,181.025692,1.646245,2.242401e+03,902.695652
std,7.048941,137.026719,3265.645077,324.706912,60.224401,1.620877e+05,34944.817084,2028.716248,937.602696,33.621543,590.963604,1.031847,4.969869e+04,2579.989246
min,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+01,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,-1.000000e+00,0.000000
25%,0.000000,0.000000,14.000000,1.000000,0.000000,1.132250e+03,33.250000,30.250000,29.250000,1.000000,13.000000,1.000000,0.000000e+00,91.750000
50%,1.000000,0.000000,52.000000,6.000000,0.000000,3.230500e+03,138.000000,77.500000,79.000000,5.000000,39.000000,1.000000,0.000000e+00,242.500000
75%,5.000000,4.000000,206.750000,26.750000,3.000000,1.194575e+04,1044.750000,264.250000,212.500000,11.750000,110.000000,2.000000,1.750000e+00,646.750000
max,66.000000,1518.000000,35665.000000,3762.000000,757.000000,2.969367e+06,723961.000000,26570.000000,11503.000000,288.000000,7907.000000,9.000000,1.117948e+06,31467.000000


In [75]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Project_ID               506 non-null    object
 1   EndOfLifeFramework       506 non-null    int64 
 2   ForFrequency             506 non-null    int64 
 3   IfFrequency              506 non-null    int64 
 4   ForEachFrequency         506 non-null    int64 
 5   WhileFrequency           506 non-null    int64 
 6   CodeLines                506 non-null    int64 
 7   CommentLines             506 non-null    int64 
 8   MethodNumber             506 non-null    int64 
 9   ClassNumber              506 non-null    int64 
 10  InterfaceNumber          506 non-null    int64 
 11  InheritanceDeclarations  506 non-null    int64 
 12  InheritanceDepth         506 non-null    int64 
 13  ExternalAPICalls         506 non-null    object
 14  HttpClientCalls          506 non-null    i

In [76]:
dataframe.head()

,Project_ID,EndOfLifeFramework,ForFrequency,IfFrequency,ForEachFrequency,WhileFrequency,CodeLines,CommentLines,MethodNumber,ClassNumber,InterfaceNumber,InheritanceDeclarations,InheritanceDepth,ExternalAPICalls,HttpClientCalls,CodeDuplication,UsingsNumber,ClassCouplingListing
0,1c3ff27b-f3a2-4a3d-a1ed-08ab3b314d91,1,0,54,2,0,1622,188,48,42,1,16,1,{},0,"[10.55, 0.0, 1.36, 0.08, 0.46, 0.17, 50.23, 0....",104,"[0, 1, 8, 1, 34, 17, 4, 16, 6, 65, 2, 46, 6, 1..."
1,c429ecce-c898-45d9-962a-44c3d6ddf0ef,0,89,10462,1279,47,279209,79805,4271,2105,38,1453,4,"{'FluentAssertions': 84, 'Azure.Extensions.Asp...",52,"[0.82, 0.0, 1.14, 0.06, 0.8, 0.3, 0.16, 0.4, 0...",8186,"[8, 2, 54, 6, 7, 7, 3, 13, 8, 9, 65, 13, 19, 1..."
2,b0c370ee-fbe8-4f95-a8eb-6fe3d188a752,1,0,70,10,0,2994,234,80,83,4,39,1,"{'Microsoft.Extensions.Caching.Memory': 1, 'Mi...",0,"[1.38, 0.15, 3.65, 1.69, 0.09, 2.07, 0.17, 0.1...",281,"[2, 58, 24, 5, 5, 1, 4, 7, 2, 2, 3, 16, 27, 3,..."
3,875f41f0-bdc5-4dc1-bced-f479a5370838,3,0,44,2,0,1127,26,44,22,1,12,1,"{'Microsoft.AspNetCore.Razor.Design': 0, 'Micr...",0,"[13.76, 2.4, 2.28, 51.3, 4.33, 18.26, 12.9, 0....",111,"[9, 7, 0, 5, 9, 4, 6, 6, 5, 18, 23, 23, 2, 3, ..."
4,b2fe1aee-78d2-4ed6-b40f-b3d251a8094f,0,43,1436,221,12,48445,15445,1436,617,93,323,3,{},0,"[0.75, 0.96, 1.87, 0.98, 9.85, 2.99, 2.69, 1.1...",1802,"[20, 17, 11, 113, 22, 12, 6, 20, 12, 7, 9, 15,..."


# Data Exploration

In [77]:
dfcopy = dataframe.copy()


In [78]:
def global_boundaries(data):
    global_max = 0
    global_min = 9999 

    for row in data:
        row_list = ast.literal_eval(row)
        row_number = [int(x) for x in row_list]
        row_max = np.max(row_number)
        row_min = np.min(row_number)
        if row_max > global_max:
            global_max = row_max
        if row_min < global_min:
            global_min = row_min

    return global_max,global_min

print(global_boundaries(dfcopy['ClassCouplingListing']))
print(global_boundaries(dfcopy['CodeDuplication']))

(6857, 0)
(100, 0)
